创建人工数据集，并存储在csv文件中
然后在经过数据处理将其转换为Tensor数据集

In [2]:
import os
import pandas as pd
import torch

In [9]:
os.makedirs(os.path.join('..','data'),exist_ok=True)
data_file = os.path.join('..','data','house_tiny.csv')
with open(data_file,'w') as f:
    f.write('NumRooms,Alley,Price\n') # 列名
    f.write('NA,Pave,127500\n') # 每行表示一个数据样本
    f.write('2,NA,106000\n')
    f.write('4,NA,178100\n')
    f.write('NA,NA,140000\n')

In [10]:
data = pd.read_csv(data_file)
data

,NumRooms,Alley,Price
0,NaN,Pave,127500
1,2.0,NaN,106000
2,4.0,NaN,178100
3,NaN,NaN,140000


In [19]:
inputs, outputs = data.iloc[:,0:2], data.iloc[:,2]

inputs["NumRooms"] = inputs["NumRooms"].fillna(inputs["NumRooms"].mean()) 
# 对于数值型缺失值，用均值替换缺失值

inputs = pd.get_dummies(inputs, dummy_na=True).astype(int)
# 对于类别型缺失值，增加一个新特征

print(inputs) 

# 经过上述处理后，就可以作为张量输入到模型中进行训练了

   NumRooms  Alley_Pave  Alley_nan
0         3           1          0
1         2           0          1
2         4           0          1
3         3           0          1


为什么你的结果是 True/False，而别人是 0/1
这是因为 pd.get_dummies() 函数在处理分类变量时，默认生成的是布尔型（bool）的哑变量，而有些教程或代码会将其转换为整数（int）的 0/1。
True/False：这是 pandas 的默认行为，True 代表该类别存在，False 代表不存在。
0/1：这是将布尔值显式转换为整数的结果，1 对应 True，0 对应 False

In [21]:
x, y = torch.tensor(inputs.values),torch.tensor(outputs.values)
x, y

(tensor([[3, 1, 0],
         [2, 0, 1],
         [4, 0, 1],
         [3, 0, 1]], dtype=torch.int32),
 tensor([127500, 106000, 178100, 140000]))

In [3]:
a = torch.arange(12)
b = a.reshape(3,4)
a, b

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11]),
 tensor([[ 0,  1,  2,  3],
         [ 4,  5,  6,  7],
         [ 8,  9, 10, 11]]))

In [4]:
b[:] = 100
a, b

# 从这里可以看到，a和b共享内存，因此修改b的值会影响a的值。

(tensor([100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]),
 tensor([[100, 100, 100, 100],
         [100, 100, 100, 100],
         [100, 100, 100, 100]]))

这是因为在 PyTorch 中，reshape() 操作默认是视图（view）操作，而不是创建新的张量。
核心原因
a = torch.arange(12) 创建了一个一维张量，占用了一块连续的内存。
b = a.reshape(3, 4) 并没有为 b 分配新的内存，而是让 b 成为 a 的一个 “视图”，也就是共享同一块内存，只是改变了数据的形状和访问方式。
当你执行 b[:] = 100 时，修改的是这块共享内存中的数据，所以 a 和 b 都会看到变化。

In [5]:
# 如果你希望 b 是一个独立的张量，修改它不会影响 a，可以使用以下方法：
# b = torch.tensor(a.reshape(3, 4))
# 这样a与b就会独立了，修改b的值不会影响a的值。